<a href="https://colab.research.google.com/github/YonghwanYim/Applied_Artificial_Intelligence/blob/master/2018_11_12_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive # Google Drive 인증
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
temp = drive.CreateFile({'id' : '1hPFSjrJOnbuWtSZguJy-tNgsq0nlGs64'}) # Generate File
temp.GetContentFile('mnist.py') # Rename

In [0]:
temp = drive.CreateFile({'id' : '1rElYJbh1CcfkjMb4Tdi_ZxvTXw_IQbK0'}) # Generate File
temp.GetContentFile('sample_weight.pkl') # Rename

In [5]:
!ls # 생성된 파일 확인

adc.json  mnist.py  sample_data  sample_weight.pkl


In [74]:
from mnist import load_mnist
import numpy as np
from PIL import Image
from IPython.display import display
from collections import OrderedDict

class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

    
class Affine:
    def __init__(self, W, b):
        self.W =W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)
        return dx

    
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None 
        self.t = None 
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size:
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx


# class Relu:
#   def __init__(self):
#     self.mask = None
    
#   def forward(self, x):
#     self.mask = (x <= 0)
#     out = x.copy()
#     out[self.mask] = 0
    
#     return out
  
#   def backward(self, dout):
#     dout[self.mask] = 0

    
# class Affine:
#   def __init__(self, W, b):
#     self.W = W
#     self.b = b
    
#     self.x = None
#     self.original_x_shape = None
#     self.dW = None
#     self.db = None
    
#   def forward(self, x):
#     self.original_x_shape = x.shape
#     x = x.reshape(x.shape[0], -1)
#     self.x = x
    
#     out = np.dot(self.x, self.W) + self.b
    
#     return out
  
#   def backward(self, dout):
#     dx = np.dot(dout, self.W.T)   # Type Error - (Non Type, float type) operator '*'
#     self.dW = np.dot(self.x.T, dout)
#     self.db = np.sum(dout, axis = 0)
    
#     dx = dx.reshape(*self.original_x_shape)
#     return dx

  
# class SoftmaxWithLoss:
#   def __init__(self):
#     self.loss = None
#     self.y = None
#     self.t = None
    
#   def forward(self, x, t):
#     self.t = t
#     self.y = softmax(x)
#     self.loss = cross_entropy_error(self.y, self.t)
    
#     return self.loss
  
#   def backward(self, dout = 1):
#     batch_size = self.t.shape[0]
#     if self.t.size == self.y.size:
#       dx = (self.y - self.t) / batch_size
#     else:
#       dx = self.y.copy()
#       dx[np.arange(batch_size), self.t] -= 1
#       dx = dx / batch_size
      
#     return dx
    

def img_show(img):
  pil_img = Image.fromarray(np.uint8(img))
  display(pil_img)
(x_train, t_train), (x_test, t_test) = load_mnist(normalize = False, flatten = True)

img = x_train[0]
label = t_train[0]
print(label)

print(img.shape)
img = img.reshape(28, 28)
print(img.shape)

img_show(img)

5
(784,)
(28, 28)


In [75]:
import pickle

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def softmax(x):
  if x.ndim == 2:
    x = x.T
    x = x - np.max(x, axis = 0)
    y = np.exp(x) / np.sum(np.exp(x), axis = 0)
    return y.T
  
  x = x - np.max(x)
  return np.exp(x) / np.sum(np.exp(x))

def get_data():
  (x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, flatten = True, one_hot_label = False)
  return x_test, t_test

def init_network():
  with open("sample_weight.pkl", "rb") as f:
    network = pickle.load(f)
  return network

def predict(network, x):
  W1, W2, W3 = network['W1'], network['W2'], network['W3']
  b1, b2, b3 = network['b1'], network['b2'], network['b3']
  
  a1 = np.dot(x, W1) + b1
  z1 = sigmoid(a1)
  a2 = np.dot(z1, W2) + b2
  z2 = sigmoid(a2)
  a3 = np.dot(z2, W3) + b3
  y = softmax(a3)
  
  return y

x, t = get_data()
network = init_network()

batch_size = 100
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
  x_batch = x[i:i+batch_size]
  y_batch = predict(network, x_batch)
  p = np.argmax(y_batch, axis = 1)
  accuracy_cnt += np.sum(p == t[i:i+batch_size])
  
  #y = predict(network, x[i])
  #p = np.argmax(y)
  #if p == t[i]:
  #  accuracy_cnt += 1

print("Accuracy: " + str(float(accuracy_cnt) / len(x)))

Accuracy: 0.9352


In [76]:
x,_ = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']
x.shape

(10000, 784)

In [77]:
x[0].shape

(784,)

In [78]:
W1.shape

(784, 50)

In [79]:
W2.shape

(50, 100)

In [80]:
W3.shape

(100, 10)

# **2층 신경망 클래스 구현**

In [0]:
class TwoLayerNet:
  
  def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
    # 가중치 초기화
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)
    #print(self.params['W1'])
    #print(self.params['b1'])
    #print(self.params['W2'])
    #print(self.params['b2'])
    
    # 계층 생성
    self.layers = OrderedDict()
    self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
    self.layers['Relu1'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
    
    self.lastLayer = SoftmaxWithLoss()
  
  
#   def predict(self, x):
#     W1, W2 = self.params['W1'], self.params['W2']
#     b1, b2 = self.params['b1'], self.params['b2']
    
#     a1 = np.dot(x, W1) + b1
#     z1 = sigmoid(a1)
#     a2 = np.dot(z1, W2) + b2
#     y = softmax(a2)
    
#     return y
  
  def predict(self, x):
    for layer in self.layers.values():
      x = layer.forward(x)
    
    return x
  
  def accuracy(self, x, t):
    y = self.predict(x)
    #print("y.shape" + str(y.shape))
    y = np.argmax(y, axis = 1)
    #print("y.shape" + str(y.shape))
    accuracy = np.sum(y == t) / float(x.shape[0])
    return accuracy
  
  def loss(self, x, t):
    y = self.predict(x)
    
    return self.lastLayer.forward(y, t)
  
  def numerical_gradient(self, x, t):
    loss_W = lambda W: self.loss(x, t) # 람다 문법으로 함수 간단하게 선언 (익명함수)
    
    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])  # numerical_gradient는 함수 밖에 정의된 것임.
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])  # class 안에서 자신의 함수를 부를때는 self를 붙여줘야 함.
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
    
    return grads
  
  def gradient(self, x, t):
    # forward
    self.loss(x, t)
    
    # backward
    dout = 1
    dout = self.lastLayer.backward(dout)
    
    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
      dout = layer.backward(dout)
    
    grads = {}
    grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
    grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
    
    return grads

In [82]:
x, t = get_data()
#network = init_network()
network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)


# accuracy_cnt = 0
# for i in range(0, len(x)):
#   y = network.predict(x[i])
#   p = np.argmax(y)
#   if p == t[i]:
#     accuracy_cnt += 1

#print("x.shape" + str(x.shape))
#print("t.shape" + str(t.shape))
print("Accuracy:" + str(network.accuracy(x, t)))

x.shape(10000, 784)
t.shape(10000,)
y.shape(10000, 10)
y.shape(10000,)
Accuracy:0.1007


# Add Loss **Function**

In [0]:
def cross_entropy_error(y, t):
  if y.ndim == 1:
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)
    
  if t.size == y.size:
    t = t.argmax(axis = 1)
    
  batch_size = y.shape[0]
  return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [0]:
def numerical_gradient(f, x):
  h = 0.0001
  grad = np.zeros_like(x)
  
  it = np.nditer(x, flags = ['multi_index'], op_flags = ['readwrite'])
  while not it.finished:
    idx = it.multi_index
    tmp_val = x[idx]
    x[idx] = float(tmp_val) + h
    fxh1 = f(x) # f(x+h)
    
    x[idx] = float(tmp_val) - h
    fxh2 = f(x) # f(x-h)
    grad[idx] = (fxh1 - fxh2) / (2 * h)
    
    x[idx] = tmp_val
    it.iternext()
    
  return grad

In [0]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, flatten = True, one_hot_label = False)

train_loss_list = []

# Hyper parameter
iters_num = 10000 # 반복횟수
train_size = x_train.shape[0]
batch_size = 100 # 미니배치 크기
learning_rate = 0.1

network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

from datetime import datetime

for i in range(iters_num):
  
  #print("i: " + str(i) + ", " + "iters_num: " + str(iters_num))
  
  # 미니배치 획득
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]
  
  # 기울기 계산 (경사하강법) 소요 시간 (Elapsed time) 측정
  t_start = datetime.now()
  
  # 기울기 계산
  #print("Under calculating each gradient matrix of loss function corresponding to given parameters (W1, b1, W2, b2)...")
  #grad = network.numerical_gradient(x_batch, t_batch)
  grad = network.gradient(x_batch, t_batch)
  #print("Calculated!")
  
  t_end = datetime.now()
  
  #print("Elapsed time per iter: " + str((t_end - t_start) * (i + 1)))
  #print("Expected left time: " + str((t_end - t_start) * (iters_num - i - 1)))
  
  # 매개변수 갱신
  for key in ('W1', 'b1', 'W2', 'b2'):
    network.params[key] -= learning_rate * grad[key]
  #print("Parameters(W1, b1, W2, b2) are adjusted better!")
  
  # 학습 경과 출력
  #print("loss: " + str(network.loss(x_batch, t_batch)))
  print("Accuracy_train: " + str(network.accuracy(x_train, t_train)))
  print("Accuracy_test: " + str(network.accuracy(x_test, t_test)))
  print()

y.shape(60000, 10)
y.shape(60000,)
Accuracy_train: 0.10283333333333333
y.shape(10000, 10)
y.shape(10000,)
Accuracy_test: 0.0961

y.shape(60000, 10)
y.shape(60000,)
Accuracy_train: 0.16263333333333332
y.shape(10000, 10)
y.shape(10000,)
Accuracy_test: 0.1638

y.shape(60000, 10)
y.shape(60000,)
Accuracy_train: 0.11705
y.shape(10000, 10)
y.shape(10000,)
Accuracy_test: 0.1174

y.shape(60000, 10)
y.shape(60000,)
Accuracy_train: 0.11585
y.shape(10000, 10)
y.shape(10000,)
Accuracy_test: 0.1162

y.shape(60000, 10)
y.shape(60000,)
Accuracy_train: 0.13686666666666666
y.shape(10000, 10)
y.shape(10000,)
Accuracy_test: 0.138

y.shape(60000, 10)
y.shape(60000,)
Accuracy_train: 0.11596666666666666
y.shape(10000, 10)
y.shape(10000,)
Accuracy_test: 0.1157

y.shape(60000, 10)
y.shape(60000,)
Accuracy_train: 0.12786666666666666
y.shape(10000, 10)
y.shape(10000,)
Accuracy_test: 0.1278

y.shape(60000, 10)
y.shape(60000,)
Accuracy_train: 0.12891666666666668
y.shape(10000, 10)
y.shape(10000,)
Accuracy_test: 0